In [3]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Load the model
model = load_model('keras_model.h5')

#membuat data dengan dimensi N (jumlah), H (tinggi), W(width), C(channel) --> RGB
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# membuka image yang akan kita klasifikasi
image = Image.open('masker.jpg')

#resize the image to a 224x224
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)

#turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image --> 0 - 225 --> -1 sampai 1
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

# Load the image into the array
data[0] = normalized_image_array

# run the inference
prediction = model.predict(data)
print(prediction)

[[0.94951016 0.05048987]]


# Tweaking the code

In [4]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Load the model
model = load_model('keras_model.h5')
#membuat data dengan dimensi N (jumlah), H (tinggi), W(width), C(channel) --> RGB
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
# membuka image yang akan kita klasifikasi
image = Image.open('masker.jpg')
#resize the image to a 224x224
size = (224, 224)
image = ImageOps.fit(image, size, Image.ANTIALIAS)
#turn the image into a numpy array
image_array = np.asarray(image)
# Normalize the image --> 0 - 225 --> -1 sampai 1
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
# Load the image into the array
data[0] = normalized_image_array
# run the inference
prediction = model.predict(data)
print(prediction)

[[0.94951016 0.05048987]]


## mencari index maximum

In [5]:
prediction

array([[0.94951016, 0.05048987]], dtype=float32)

In [7]:
img_class = {0:"bermasker", 1:"tidak bermasker"}
img_class[np.argmax(prediction)]

'bermasker'

In [11]:
"{0:.2f}".format(np.max(prediction))

'0.95'

# Wrapping menjadi satu fungsi

In [14]:
# Load the model
model = load_model('keras_model.h5')

def classify_image(img_path): 
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32) #membuat data dengan dimensi N (jumlah), H (tinggi), W(width), C(channel) --> RGB
    image = Image.open('masker.jpg') # membuka image yang akan kita klasifikasi

    #resize the image to a 224x224
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)
    image_array = np.asarray(image) #turn the image into a numpy array

    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1 # Normalize the image --> 0 - 225 --> -1 sampai 1

    data[0] = normalized_image_array # Load the image into the array
    img_class = {0:"bermasker", 1:"tidak bermasker"}
    prediction = model.predict(data) # run the inference
    
    return(img_class[np.argmax(prediction)], "{0:.2f}".format(np.max(prediction)))

In [17]:
img_path = "masker.jpg"
classify_image(img_path)

('bermasker', '0.95')

# Realtime classification

In [21]:
import cv2
cap = cv2.VideoCapture(0) #memperbolehkan cv2 mengambil gambar
print(cap.get(3), cap.get(4))

while True:
    ret, frame = cap.read() #return gambar yang diambil oleh opencv 
    if ret == True:
        cv2.imshow("prediction", frame) #menampilkan gambar
        
    if cv2.waitKey(1) & 0xFF == ord('q'): #close window tekan "q"
        break
        
cap.release()
cv2.destroyAllWindows()

640.0 480.0


## resize 640 x 480 --> 224 x 224

In [24]:
import cv2
cap = cv2.VideoCapture(0) #memperbolehkan cv2 mengambil gambar
print(cap.get(3), cap.get(4))
w = cap.get(3)
h = cap.get(4)

a_w = 224/w
a_h = 224/h

while True:
    ret, frame = cap.read() #return gambar yang diambil oleh opencv 
    if ret == True:
        frame_to_clf = cv2.resize(frame, None, fx= a_w, fy = a_h)
        cv2.imshow("prediction", frame_to_clf) #menampilkan gambar
        
    if cv2.waitKey(1) & 0xFF == ord('q'): #close window tekan "q"
        break
        
cap.release()
cv2.destroyAllWindows()

640.0 480.0


# Menggabungkan fungsi klasifikasi w/ AI + OpenCV

In [8]:
from keras.models import load_model
from PIL import Image, ImageOps
import numpy as np

# Load the model
model = load_model('keras_model.h5')

def classify_image(img_path): 
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32) #membuat data dengan dimensi N (jumlah), H (tinggi), W(width), C(channel) --> RGB
    image = img_path
    image_array = np.asarray(image) #turn the image into a numpy array
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1 # Normalize the image --> 0 - 225 --> -1 sampai 1
    data[0] = normalized_image_array # Load the image into the array
    img_class = {0:"bermasker", 1:"tidak bermasker"}
    prediction = model.predict(data) # run the inference
    return(img_class[np.argmax(prediction)], "{0:.2f}".format(np.max(prediction)))

import cv2
cap = cv2.VideoCapture(0) #memperbolehkan cv2 mengambil gambar
print(cap.get(3), cap.get(4))
w = cap.get(3)
h = cap.get(4)

a_w = 224/w
a_h = 224/h

def draw(text):
    '''
    function untuk menambahkan hasil klasifikasi ke realtime image footage
    '''
    font = cv2.FONT_ITALIC
    if text[0] == "bermasker":
        text = str(text)
        cv2.putText(frame, text, (20,40), font, 1, (255,0,0), 1, cv2.LINE_AA)
    else:
        text = str(text)
        cv2.putText(frame, text, (20,40), font, 1, (0,0, 238), 1, cv2.LINE_AA)

while True:
    ret, frame = cap.read() #return gambar yang diambil oleh opencv 
    if ret == True:
        frame_to_clf = cv2.resize(frame, None, fx= a_w, fy = a_h)
        prediction = classify_image(frame_to_clf)
        print("\r" + str(prediction), end="  ")
        
        draw(prediction)
        cv2.imshow("prediction", frame) #menampilkan gambar
        
    if cv2.waitKey(1) & 0xFF == ord('q'): #close window tekan "q"
        break
        
cap.release()
cv2.destroyAllWindows()

640.0 480.0
('tidak bermasker', '1.00')    